# b.     STORING THE DATA

#### ALL REQUIRED IMPORTS

In [3]:
import os
import csv
import requests
from pprint import pprint
import json
import string
import glob
import shutil

In [4]:
directory = os.path.dirname(os.getcwd())
directory
cur_path= directory+'/'
cur_path

'/Users/sonalichaudhari/Desktop/FINAL_Python/'

In [51]:
key = os.environ.get('b')
print(key)

08b67039063a05b26010cb41df61d837


In [15]:
#id_list = []

with open(cur_path+'extra/IDS.csv', newline='') as f:
                reader = csv.reader(f)
                next(reader, None)
                try:
                    for record in reader:
                        c = record[0]
                        #print(c)
                        if c not in id_list:
                            id_list.append(c)                       
                except csv.Error as e:
                    sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))

In [20]:
id_list = [int(x) for x in id_list]
print('Number of UNIQUE City Ids :',len(id_list))

Number of UNIQUE City Ids : 1643


#### COLLECTING RESTAURANT DATA PER CITY ID

In [54]:
#result=[]

header1={'user-key':key}

for f in id_list[400:800]:
    try:
        v = str(f)
        url1 = 'https://developers.zomato.com/api/v2.1/search?entity_id='+v+'&entity_type=city&start=20'
        result.append(requests.get(url1,headers=header1).json())
    except ValueError:
        continue

In [97]:
with open('/Users/sonalichaudhari/Desktop/FINAL_Python/extra/ResultCounts.csv', 'a') as csvfile:    
    fieldnames = ['ID', 'ResultCounts']  #Naming the title Row of the CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for j in range(533):#len(result)-1):
        cid = id_list[j]
        counts = result[j]['results_found']
        writer.writerow({'ID':cid,'ResultCounts':counts}) 

In [109]:
COUNTRY={1:'India',14:'Australia',216:'United States',37:'Canada',123:'Malaysia',215:'United Kingdom',\
         54:'Czech Republic', 30: 'Brasil',42:'Chile',94:'Indonesia',97:'Ireland',99:'Italy',\
        112:'Lebanon',148:'New Zealand', 162:'Philippines',163:'Poland',164:'Portugal',166:'Qatar',\
        184:'Singapore',185:'Slovakia', 189:'South Africa',191:'Sri Lanka', 208:'Turkey',214:'UAE',}
with open('COUNTRY.json', 'w') as outfile:
    json.dump(COUNTRY, outfile)
len(COUNTRY)

24

In [107]:
#Storing
final=[]
for i in range(532):#len(result)):
    for j in range(len(result[i]['restaurants'])):
        if result[i]['restaurants'][j] not in final:
            final.append(result[i]['restaurants'][j])
            
len(final)

9735

In [110]:
#APPENDING COUNTRY NAME TO EVERY RESTAURANT JSON FILE
for i in range(len(final)):
    countryid=final[i]['restaurant']['location']['country_id']
    if(countryid in COUNTRY.keys()):
        h=COUNTRY[countryid]
        final[i]['restaurant']['location']['country_name']=h

In [ ]:
# LOADING THE JSON FILES IN TEMPORARY FOLDER
p = cur_path+'/TEMP/'
for y in final:
    rest = y['restaurant']['name'].replace('/',' ')+'_'+str(y['restaurant']['R']['res_id'])
    with open(p + rest + str(datetime.now()) +'.json', 'w',encoding="utf-8") as outfile:
                        json.dump(y, outfile, ensure_ascii=False)

In [364]:
#final[1748]
#1836
#3026

In [6]:
def read_files(file):      # Create a function to read files. 
    with open (file, encoding='utf-8', errors="ignore") as f:   # Need to deal with unicode problem.
        content = json.load(f)
        hierarchy_list = {'country': content['restaurant']['location']['country_name'],\
                          'city': content['restaurant']['location']['city'],\
                          'name': content['restaurant']['name']}
        # Remove the ',' in some names for later output csv file.
        return hierarchy_list

In [7]:
def file_distribution(dict, file):   # Generate hierarchy folder structure and dump json file
    file_name = '/Users/sonalichaudhari/Desktop/FINAL_Python/Data/'\
    +'/' + dict['country'] + '/' + dict['city'] + '/' + dict['name'] \
                + '.json'
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    #fp = open (file_name, 'w')
    #with open (file, 'r', encoding='utf-8', errors="ignore") as jfile:
        #fp.write(json.dumps(json.load(jfile)))      # Try to find out the difference between dump and dumps
    shutil.move(file,file_name)

In [ ]:
for json_file in glob.glob("/Users/sonalichaudhari/Desktop/FINAL_Python/TEMP/*.json"):
    file_distribution(read_files(json_file),json_file)